<a href="https://colab.research.google.com/github/rakshitrk/handwriten_digit_recogination/blob/master/convotution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch 
import torchvision as tv
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

In [2]:
train_cuda = torch.cuda.is_available()

if not train_cuda:
  print('CUDA not available. Training on CPU.')
else:
  print('CUDA is available. Training on GPU.')

CUDA is available. Training on GPU.


In [3]:
train_data = tv.datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_data = tv.datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor(),download=True)

Processing...
Done!


In [0]:
#spliting test data to validetion data

size = len(train_data)
indices = list(range(size))
split = int(np.floor(0.2 * size))
np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

train = SubsetRandomSampler(train_indices)
val = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = 20, sampler = train)
val_loader = torch.utils.data.DataLoader(train_data, batch_size = 20, sampler = val)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 20)

In [12]:
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.conv1 = nn.Conv2d(1,16,3)
    self.conv2 = nn.Conv2d(16,32,3)
    self.pool = nn.MaxPool2d(2,2)
    self.fc1 = nn.Linear(32*4*4,500)
    self.fc2 = nn.Linear(500,10)
    self.drop = nn.Dropout(0.2)
    
  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1,32*4*4)
    x = self.drop(x)
    x = F.relu(self.fc1(x))
    x = self.drop(x)
    x = F.relu(self.fc2(x))
    return x
  
m = Model()
print(m)

if train_cuda:
  m.cuda()
  
    

Model(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (drop): Dropout(p=0.2)
)


In [0]:
critersion = nn.CrossEntropyLoss()
optimizer = optim.SGD(m.parameters(), lr = 0.01)
